Boilerplate for setup:

In [3]:

# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2

In [4]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import itertools

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer, HookedTransformerConfig

import oocl

/home/scur1199/.conda/envs/dl2023/lib/python3.11/site-packages/torch/distributed/_functional_collectives.py:22: UserWarning: Unable to import torchdynamo util `is_torchdynamo_compiling`, so won't support torchdynamo correctly
  warnings.warn(


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'set_guard_fail_hook' from 'torch._dynamo.eval_frame' (/home/scur1199/.conda/envs/dl2023/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py)

In [ ]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We load a saved model at two different checkpoints and provide the subsets used in X2. Model_1 used below is post-X1, pre-X2 training and model_2 is 500 steps into X2 training.

In [ ]:
model_path = "oocl_120_step_9999.pt"

mod = oocl.DataParams.mod

new_transformer_config = oocl.transformer_config
new_transformer_config.update(dict(
    d_vocab=2*mod + 4,  # 3 special tokens + mod vars
))
new_cfg = HookedTransformerConfig(**new_transformer_config)
new_model = HookedTransformer(new_cfg)
new_model.load_state_dict(torch.load(model_path, map_location=device))
new_model.eval()
model_1 = new_model

model_path = "oocl_120_step_10500.pt"

mod = oocl.DataParams.mod

new_transformer_config = oocl.transformer_config
new_transformer_config.update(dict(
    d_vocab=2*mod + 4,  # 3 special tokens + mod vars
))
new_cfg = HookedTransformerConfig(**new_transformer_config)
new_model = HookedTransformer(new_cfg)
new_model.load_state_dict(torch.load(model_path, map_location=device))
new_model.eval()
model_2 = new_model

Dt3 = [103, 100, 19, 63, 99, 21, 79, 18, 59, 36, 16, 39, 20, 55, 57, 110, 52, 81, 10, 11, 106, 61, 94, 69, 58, 0, 4, 73, 67, 3]
Df4 = [92, 117, 98, 118, 84, 33, 113, 44, 116, 91, 101, 53, 56, 24, 43, 72, 62, 5, 75, 83, 114, 27, 102, 77, 32, 93, 42, 29, 7, 68]

Let's look at how the question accuracy evolves over X2 

In [ ]:
from oocl import create_questions, evaluate
from torch.utils.data import DataLoader

Dt3_questions = {}
Df4_questions = {}

Dt3_acc_1 = {}
Df4_acc_1 = {}

Dt3_acc_2 = {}
Df4_acc_2 = {}

questions = {}

for num in Dt3:

    Dt3_questions[num] = create_questions([num])
    Dt3_acc_1[num] = evaluate(model_1, DataLoader(Dt3_questions[num].unsqueeze(0)), device)[0]

    # now drill down to individual questions hackily

    for q in Dt3_questions[num]:
        questions[q] = evaluate(model_1, DataLoader(q.unsqueeze(0).unsqueeze(0)), device)[0]

for num in Df4:

    Df4_questions[num] = create_questions([num])
    Df4_acc_1[num] = evaluate(model_1, DataLoader(Df4_questions[num].unsqueeze(0)), device)[0]

    # now drill down to individual questions hackily

    for q in Df4_questions[num]:

        questions[q] = evaluate(model_1, DataLoader(q.unsqueeze(0).unsqueeze(0)), device)[0]

for num in Dt3:

    Dt3_questions[num] = create_questions([num])
    Dt3_acc_2[num] = evaluate(model_2, DataLoader(Dt3_questions[num].unsqueeze(0)), device)[0]

for num in Df4:

    Df4_questions[num] = create_questions([num])
    Df4_acc_2[num] = evaluate(model_2, DataLoader(Df4_questions[num].unsqueeze(0)), device)[0]
